# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle
from sklearn.model_selection import train_test_split

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'data/train.p'
validation_file= 'data/valid.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
x_train, y_train = train['features'], train['labels']
x_valid, y_valid = valid['features'], valid['labels']
x_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = len(x_train)

# TODO: Number of validation examples
n_validation = len(x_valid)

# TODO: Number of testing examples.
n_test = len(x_test)

# TODO: What's the shape of an traffic sign image?
image_shape = x_train.shape[1:3]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import numpy as np
import random
import matplotlib.pyplot as plt
import cv2
# Visualizations will be shown in the notebook.
%matplotlib inline

#for i,j in enumerate(y_train):
#    if j==38:
#        im = x_train[i]
#        plt.imshow(im)

index = random.randint(0, len(x_train))
im = x_train[index]
plt.imshow(im)
print("Label ID is:", y_train[index])

In [ ]:
# Get histograms of the images labels
train_hist = np.histogram(y_train, bins = 43)
valid_hist = np.histogram(y_valid, bins = 43)
test_hist = np.histogram(y_test, bins = 43)

# Generating bin centers
bin_centers = range(43)

# Plot histograms.
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize = (10, 10))

ax1.set_title('Training Dataset')
ax1.set_xlabel('Label ID')
ax1.set_ylabel('Num of Examples')
ax1.set_xlim(-0.5, 42.5)
ax1.set_xticks(range(43))
ax1.bar(bin_centers, train_hist[0])

ax2.set_title('Validation Dataset')
ax2.set_xlabel('Label ID')
ax2.set_ylabel('Num of Examples')
ax2.set_xlim(-0.5, 42.5)
ax2.set_xticks(range(43))
ax2.bar(bin_centers, valid_hist[0])

ax3.set_title('Test Dataset')
ax3.set_xlabel('Label ID')
ax3.set_ylabel('Num of Examples')
ax3.set_xlim(-0.5, 42.5)
ax3.set_xticks(range(43))
ax3.bar(bin_centers, test_hist[0])

fig.tight_layout()
plt.show()

fig.savefig('datasets_vis.jpg')

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
from sklearn.utils import shuffle
import cv2

# Define a function to convert an array of color images to grayscale.
def rgb2gray(x):
    gray = np.zeros((len(x), 32, 32), dtype = np.uint8)
    for i in range(len(x)):
        gray[i] = cv2.cvtColor(x[i], cv2.COLOR_RGB2GRAY)
    return np.expand_dims(gray, axis = 3)

# Define a function to normalize an array of images.
def normalize(x):
    return (x.astype(float) - 128) / 128

# Convert images to grayscale.
x_train_gray = rgb2gray(x_train)
x_valid_gray = rgb2gray(x_valid)
x_test_gray = rgb2gray(x_test)

# Normalize images.
x_train_norm = normalize(x_train_gray)
x_valid_norm = normalize(x_valid_gray)
x_test_norm = normalize(x_test_gray)

In [ ]:
# Visualize images after preprocessing.

index = random.randint(0, len(x_train))
im = x_train[index]
im_gray = x_train_gray[index]
im_norm = x_train_norm[index]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 10))

ax1.imshow(im)
ax2.imshow(np.squeeze(im_gray, axis = 2), cmap = 'gray')

fig.savefig('colorvsgray.jpg', bbox_inches = 'tight')

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
import tensorflow as tf
from tensorflow.contrib.layers import flatten

# Define model architecture.
def LeNet(x, keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # Parameter variables.
    weights = {'wc1': tf.Variable(tf.truncated_normal([3, 3, 1, 24], mean = mu, stddev = sigma)),
               'wc2': tf.Variable(tf.truncated_normal([3, 3, 24, 36], mean = mu, stddev = sigma)),
               'wc3': tf.Variable(tf.truncated_normal([3, 3, 36, 48], mean = mu, stddev = sigma)),
               'wfc1': tf.Variable(tf.truncated_normal([768, 384], mean = mu, stddev = sigma)),
               'wfc2': tf.Variable(tf.truncated_normal([384, 192], mean = mu, stddev = sigma)),
               'wfc3': tf.Variable(tf.truncated_normal([192, 96], mean = mu, stddev = sigma)),
               'wfc4': tf.Variable(tf.truncated_normal([96, 43], mean = mu, stddev = sigma))}
    biases = {'bc1': tf.Variable(tf.zeros(24)),
              'bc2': tf.Variable(tf.zeros(36)),
              'bc3': tf.Variable(tf.zeros(48)),
              'bfc1': tf.Variable(tf.zeros(384)),
              'bfc2': tf.Variable(tf.zeros(192)),
              'bfc3': tf.Variable(tf.zeros(96)),
              'bfc4': tf.Variable(tf.zeros(43))}
              
    # Layer 1: 1st Convolution. Input = 32x32x1. Output = 30x30x24.
    strides = [1, 1, 1, 1]
    padding = 'VALID'
    conv1 = tf.nn.conv2d(x, weights['wc1'], strides, padding) + biases['bc1']

    # Activation.
    conv1act = tf.nn.relu(conv1, name = 'conv1act')

    # Pooling. Input = 30x30x24. Output = 15x15x24.
    kernel = [1, 2, 2, 1]
    strides = [1, 2, 2, 1]
    padding = 'VALID'
    pool1 = tf.nn.max_pool(conv1act, kernel, strides, padding)

    # Layer 2: 2nd Convolution. Input = 15x15x24. Output = 13x13x36.
    strides = [1, 1, 1, 1]
    padding = 'VALID'
    conv2 = tf.nn.conv2d(pool1, weights['wc2'], strides, padding) + biases['bc2']
    
    # Activation.
    conv2act = tf.nn.relu(conv2, name = 'conv2act')

    # Pooling. Input = 13x13x36. Output = 6x6x36.
    kernel = [1, 2, 2, 1]
    strides = [1, 2, 2, 1]
    padding = 'VALID'
    pool2 = tf.nn.max_pool(conv2act, kernel, strides, padding)
    
    # Layer 3: 3rd Convolution. Input = 6x6x36. Output = 4x4x48.
    strides = [1, 1, 1, 1]
    padding = 'VALID'
    conv3 = tf.nn.conv2d(pool2, weights['wc3'], strides, padding) + biases['bc3']
    
    # Activation.
    conv3act = tf.nn.relu(conv3, name = 'conv3act')

    # Flatten. Input = 4x4x48. Output = 768.
    flat = flatten(conv3act)
    
    # Layer 4: 1st Fully Connected Layer. Input = 768. Output = 384.
    fc1 = tf.nn.dropout(tf.add(tf.matmul(flat, weights['wfc1']), biases['bfc1']), keep_prob)
    
    # Activation.
    fc1act = tf.nn.relu(fc1)

    # Layer 5: 2nd Fully Connected Layer. Input = 384. Output = 192.
    fc2 = tf.nn.dropout(tf.add(tf.matmul(fc1act, weights['wfc2']), biases['bfc2']), keep_prob)
    
    # Activation.
    fc2act = tf.nn.relu(fc2)
    
    # Layer 6: 3rd Fully Connected Layer. Input = 192. Output = 96.
    fc3 = tf.nn.dropout(tf.add(tf.matmul(fc2act, weights['wfc3']), biases['bfc3']), keep_prob)
    
    # Activation.
    fc3act = tf.nn.relu(fc3)

    # Layer 7: Output Layer. Input = 96. Output = 43.
    logits = tf.add(tf.matmul(fc3act, weights['wfc4']), biases['bfc4'])
    
    return logits

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

# Input data placeholders.
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)
keep_prob = tf.placeholder(tf.float32)

# Training hyperparameters.
epochs = 10
batch_size = 128
rate = 0.001

# Training pipeline.
logits = LeNet(x, keep_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels = one_hot_y, logits = logits)
loss_function = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training = optimizer.minimize(loss_function)

In [ ]:
# Evaluation pipeline.
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(x_data, y_data):
    num_examples = len(x_data)
    accuracy_sum = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = x_data[offset:offset + batch_size], y_data[offset:offset + batch_size]
        accuracy_evaluation = sess.run(accuracy, feed_dict = {x: batch_x, y: batch_y, keep_prob: 1})
        accuracy_sum += accuracy_evaluation
        num_batches = np.ceil(num_examples / batch_size)
    return accuracy_sum / num_batches

In [ ]:
# Train the model.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(x_train)
    
    print("Training...")
    print()
    for i in range(epochs):
        x_train_norm, y_train = shuffle(x_train_norm, y_train)
        for offset in range(0, num_examples, batch_size):
            batch_x, batch_y = x_train_norm[offset:offset + batch_size], y_train[offset:offset + batch_size]
            sess.run(training, feed_dict = {x: batch_x, y: batch_y, keep_prob: 0.6})
        
        training_accuracy = evaluate(x_train_norm, y_train)
        validation_accuracy = evaluate(x_valid_norm, y_valid)
        print("Epoch {} ...".format(i + 1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './tsigns')
    print("Model saved")

In [ ]:
# Test the model.
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    test_accuracy = evaluate(x_test_norm, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

# Read, Resize, and Expand the first dimension of the images, to make each image (1, 32, 32, 3).
s1 = np.expand_dims((cv2.resize(plt.imread('data/8.jpg'), (32, 32))), axis = 0)
s2 = np.expand_dims((cv2.resize(plt.imread('data/12.jpg'), (32, 32))), axis = 0)
s3 = np.expand_dims((cv2.resize(plt.imread('data/14.jpg'), (32, 32))), axis = 0)
s4 = np.expand_dims((cv2.resize(plt.imread('data/25.jpg'), (32, 32))), axis = 0)
s5 = np.expand_dims((cv2.resize(plt.imread('data/38.jpg'), (32, 32))), axis = 0)

# Plot images.
f, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize = (20, 20))
ax1.imshow(np.squeeze(s1))
ax2.imshow(np.squeeze(s2))
ax3.imshow(np.squeeze(s3))
ax4.imshow(np.squeeze(s4))
ax5.imshow(np.squeeze(s5))

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

# Convert images to grayscale.
s1_gray = rgb2gray(s1)
s2_gray = rgb2gray(s2)
s3_gray = rgb2gray(s3)
s4_gray = rgb2gray(s4)
s5_gray = rgb2gray(s5)

# Normalize images.
s1_norm = normalize(s1_gray)
s2_norm = normalize(s2_gray)
s3_norm = normalize(s3_gray)
s4_norm = normalize(s4_gray)
s5_norm = normalize(s5_gray)

# Test the model on the 5 images dataset.
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    s1_softmax = sess.run(tf.nn.softmax(logits), feed_dict = {x: s1_norm, keep_prob: 1})
    print("Image 1 Label is:", np.argmax(s1_softmax))
    s2_softmax = sess.run(tf.nn.softmax(logits), feed_dict = {x: s2_norm, keep_prob: 1})
    print("Image 2 Label is:", np.argmax(s2_softmax))
    s3_softmax = sess.run(tf.nn.softmax(logits), feed_dict = {x: s3_norm, keep_prob: 1})
    print("Image 3 Label is:", np.argmax(s3_softmax))
    s4_softmax = sess.run(tf.nn.softmax(logits), feed_dict = {x: s4_norm, keep_prob: 1})
    print("Image 4 Label is:", np.argmax(s4_softmax))
    s5_softmax = sess.run(tf.nn.softmax(logits), feed_dict = {x: s5_norm, keep_prob: 1})
    print("Image 5 Label is:", np.argmax(s5_softmax))

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

x_internet_test = np.concatenate((s1_norm, s2_norm, s3_norm, s4_norm, s5_norm), axis = 0)
y_internet_test = np.array([8, 12, 14, 25, 38], np.uint8)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    internet_test_accuracy = evaluate(x_internet_test, y_internet_test)
    print("Internet Test Accuracy = {:.3f}".format(internet_test_accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

# Print top 5 probabilities for first image.
with tf.Session() as sess:
    output = sess.run(tf.nn.top_k(tf.constant(s1_softmax), k = 5))
    
print(output)

In [ ]:
# Print top 5 probabilities for second image.
with tf.Session() as sess:
    output = sess.run(tf.nn.top_k(tf.constant(s2_softmax), k = 5))
    
print(output)

In [ ]:
# Print top 5 probabilities for third image.
with tf.Session() as sess:
    output = sess.run(tf.nn.top_k(tf.constant(s3_softmax), k = 5))
    
print(output)

In [ ]:
# Print top 5 probabilities for fourth image.
with tf.Session() as sess:
    output = sess.run(tf.nn.top_k(tf.constant(s4_softmax), k = 5))
    
print(output)

In [ ]:
# Print top 5 probabilities for fifth image.
with tf.Session() as sess:
    output = sess.run(tf.nn.top_k(tf.constant(s5_softmax), k = 5))
    
print(output)

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function   
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

In [ ]:
# Visualize feature maps of a sign image using trained weights.

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.')) 
    
    conv1act = tf.get_default_graph().get_tensor_by_name("conv1act:0")  
    outputFeatureMap(s4_norm, conv1act, plt_num=1)
    
    conv2act = tf.get_default_graph().get_tensor_by_name("conv2act:0")  
    outputFeatureMap(s4_norm, conv2act, plt_num=2)
    
    conv3act = tf.get_default_graph().get_tensor_by_name("conv3act:0")  
    outputFeatureMap(s4_norm, conv3act, plt_num=3)

In [ ]:
# Visualize feature maps of a sign image using untrained weights.

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    conv1act = tf.get_default_graph().get_tensor_by_name("conv1act:0")  
    outputFeatureMap(s4_norm, conv1act, plt_num=1)
    
    conv2act = tf.get_default_graph().get_tensor_by_name("conv2act:0")  
    outputFeatureMap(s4_norm, conv2act, plt_num=2)
    
    conv3act = tf.get_default_graph().get_tensor_by_name("conv3act:0")  
    outputFeatureMap(s4_norm, conv3act, plt_num=3)

In [ ]:
# Visualize feature maps of a non-traffic sign image using trained weights

# Read, Resize, and Expand the first dimension of the image, to make each image (1, 32, 32, 3).
non_sign = np.expand_dims((cv2.resize(plt.imread('data/non_sign.jpg'), (32, 32))), axis = 0)

# Convert image to grayscale.
non_sign_gray = rgb2gray(non_sign)

# Normalize image.
non_sign_norm = normalize(non_sign_gray)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.')) 

    conv1act = tf.get_default_graph().get_tensor_by_name("conv1act:0")  
    outputFeatureMap(non_sign_norm, conv1act, plt_num=1)
    
    conv2act = tf.get_default_graph().get_tensor_by_name("conv2act:0")  
    outputFeatureMap(non_sign_norm, conv2act, plt_num=2)
    
    conv3act = tf.get_default_graph().get_tensor_by_name("conv3act:0")  
    outputFeatureMap(non_sign_norm, conv3act, plt_num=3)